# Conduct a Vector Query

> This topic describes how to conduct a vector query.

# Connect to milvus.

In [1]:
#r "nuget: IO.Milvus, 2.2.1-alpha.7"
#!import config/Settings.cs

using IO.Milvus;
using IO.Milvus.Client;
using IO.Milvus.Client.gRPC;
using IO.Milvus.Client.REST;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

//Connect to milvus
(string endpoint,int port,string userName,string password) = Settings.LoadFromFile();

IMilvusClient milvusClient = default;

milvusClient = new MilvusGrpcClient(endpoint,port,userName,password);
milvusClient

Installed Packages IO.Milvus, 2.2.1-alpha.7

Address,in01-a01c84443bc084c.aws-us-west-2.vectordb.zillizcloud.com:19535


# Load collection

🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺

All search and query operations within Milvus are executed in memory. Load the collection to memory before conducting a vector query.

In [3]:
string collectionName = "book";

if(!(await milvusClient.HasCollectionAsync(collectionName)))
{
    Console.WriteLine($"Collection {collectionName} not exist");
    return;
}

await milvusClient.LoadCollectionAsync(collectionName);

//Waiting for collection loaded

long progress = await milvusClient.GetLoadingProgressAsync(collectionName);

//Waiting for collection loaded, skip this code when you are using a MilvusRestClient
var progressItems = milvusClient.WaitForLoadingProgressCollectionValueAsync(
    collectionName,
    null,
    TimeSpan.FromMilliseconds(500),
    TimeSpan.FromSeconds(10));

await foreach(var progress in progressItems){
    Console.WriteLine($"{progress}%");
}

var collectionInfo = await milvusClient.DescribeCollectionAsync(collectionName);
collectionInfo

100%


IO.Milvus.DetailedMilvusCollection Aliases CollectionName book CollectionId 441943051529528722 ConsistencyLevel Session CreatedUtcTime 2023-06-05 15:08:02Z Schema CollectionSchema: {AutoId: False, Description, , Fields: 4} AutoId False Description Fields index value 0 IO.Milvus.FieldType AutoId False DataType Int64 Description <null> FieldId 100 IndexParams (empty) IsPrimaryKey True Name book_id TypeParams (empty) 1 IO.Milvus.FieldType AutoId False DataType Int64 Description <null> FieldId 101 IndexParams (empty) IsPrimaryKey False Name word_count TypeParams (empty) 2 IO.Milvus.FieldType AutoId False DataType VarChar Description <null> FieldId 102 IndexParams (empty) IsPrimaryKey False Name book_name TypeParams key value max_length 256 3 IO.Milvus.FieldType AutoId False DataType FloatVector Description <null> FieldId 103 IndexParams (empty) IsPrimaryKey False Name book_intro TypeParams key value dim 2 Name book ShardsNum 1 StartPositions (empty)

# Conduct a vector query

The following example filters the vectors with certain book_id values, and returns the book_id field and book_intro of the results.

In [4]:
string expr = "book_id in [2,4,6,8]";

var queryResult = await milvusClient.QueryAsync(
    collectionName,
    expr,
    new[] { "book_id", "word_count" });

queryResult

IO.Milvus.MilvusQueryResult CollectionName FieldsData index value 0 Field: {FieldName: word_count, DataType: Int64, Data: 4, RowCount: 4} Data [ 10002, 10004, 10006, 10008 ] RowCount 4 FieldName word_count FieldId 0 DataType Int64 1 Field: {FieldName: book_id, DataType: Int64, Data: 4, RowCount: 4} Data [ 2, 4, 6, 8 ] RowCount 4 FieldName book_id FieldId 0 DataType Int64

| Parameter | Description |
| --------- | ----------- |
| output_fields (optional) | List of names of the fields to return. |
| vectors | Vectors to query. |
| expr | Boolean expression used to filter attribute. Find more expression details in Boolean Expression Rules. |